In [9]:
import numpy as np
import cvxpy as cp
from scipy.optimize import linprog

### Задача 5.1

In [10]:
#minimize c.T @ x
#such that A_ub @ x <= b_ub
#A_eq @ x == b_eq
costs = np.array([2, 4, 3, 7, 7, 6])
b_cap = np.array([180, 220])
A_cap = np.array([[1, 1, 1, 0, 0, 0],[ 0, 0, 0, 1, 1, 1]])

b_needs = np.array([110, 150, 140])
A_needs = np.array([[1, 0, 0, 1, 0, 0],[ 0, 1, 0, 0, 1, 0],[ 0, 0, 1, 0, 0, 1]])

In [11]:
A = np.append(A_cap, A_needs, axis=0)
b = np.append(b_cap, b_needs)
print(A,b)

[[1 1 1 0 0 0]
 [0 0 0 1 1 1]
 [1 0 0 1 0 0]
 [0 1 0 0 1 0]
 [0 0 1 0 0 1]] [180 220 110 150 140]


In [12]:
c = costs

In [13]:
res = linprog(c=c, A_ub=A_cap, b_ub=b_cap, A_eq=A_needs, b_eq=b_needs)

In [14]:
res.x.round(0)

array([110.,  36.,  34.,   0., 114., 106.])

In [15]:
c.T @ res.x.round(0)

1900.0

In [17]:
x = cp.Variable(shape=c.shape[0], integer = True)
x

Variable((6,), integer=True)

In [18]:
total_costs = c.T * x
constraint_capacity = (A_cap @ x <= b_cap)
constraint_needs = (A_needs @ x == b_needs)
x_positive = (x >= 0)

C:\Users\SVasilchenko\anaconda3\lib\site-packages\cvxpy\expressions\expression.py:556: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)


In [19]:
total_costs

Expression(AFFINE, UNKNOWN, ())

In [22]:
problem = cp.Problem(cp.Minimize(total_costs), constraints=[constraint_capacity, constraint_needs, x_positive])

In [23]:
problem.solve(solver="ECOS_BB")

1899.999999997038

### Задача 5.2 о назначениях

In [ ]:
import numpy as np
import cvxpy as cp

In [46]:
costs = np.array([[1000, 12, 10, 19, 8],
                  [12, 1000, 3, 7, 2],
                  [10, 3, 1000, 6, 20],
                  [19, 7, 6, 1000, 4],
                  [8, 2, 20, 4, 1000]])

b = np.ones(5)

In [47]:
assignment = cp.Variable(shape=costs.shape, boolean = True)

#ограничения: суммы по строкам и столбцам равны единице
job_costraint = (cp.sum(assignment, axis=1) == b)
worker_costraint = (cp.sum(assignment, axis=0) == b)

In [58]:
#Оптимизируемая функция
total_costs = cp.sum(cp.multiply(costs,assignment))

#Решение
problem = cp.Problem(cp.Minimize(total_costs), constraints = [job_costraint, worker_costraint])
result = problem.solve(solver='ECOS_BB')

print(f"{assignment.value.round(0)} work cost: {result.round(0)}")

[[ 1.  0.  0.  0.  0.]
 [ 0. -0.  1.  0.  0.]
 [ 0.  1. -0.  0.  0.]
 [ 0.  0.  0. -0.  1.]
 [ 0.  0.  0.  1. -0.]] work cost: 1014.0
